<a href="https://colab.research.google.com/github/stonepreston/jetson-nano-tf-keras/blob/master/Mobile_Net_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Install Tensorflow

In [0]:
!pip install tensorflow==1.13.1
import tensorflow as tf
tf.VERSION

Mount Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Add variable to store path to drive folder

In [0]:
root_path = '/content/gdrive/My Drive/USDA/Nano_Tensorflow'

Save keras model

In [0]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as Net
model = Net(weights='imagenet')
model.save(f'{root_path}/keras_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
14540800/14536120 [==============================] - 1s 0us/step


Freeze graph

In [0]:

import tensorflow as tf
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model

tf.keras.backend.clear_session()

def freeze_graph(graph, session, output, save_pb_dir=root_path, save_pb_name='frozen_model.pb', save_pb_as_text=False):
  with graph.as_default():
    graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
    graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
    graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
    return graphdef_frozen

tf.keras.backend.set_learning_phase(0) 

model = load_model(f'{root_path}/keras_model.h5')

session = tf.keras.backend.get_session()

input_names = [t.op.name for t in model.inputs]
output_names = [t.op.name for t in model.outputs]

# Prints input and output nodes names, take notes of them.
print(input_names, output_names)

frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs], save_pb_dir=root_path)


['input_1'] ['Logits/Softmax']
Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 262 variables.
INFO:tensorflow:Converted 262 variables to const ops.


Convert for use with TensorRT

In [0]:
import tensorflow.contrib.tensorrt as trt

trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16',
    minimum_segment_size=50
)



INFO:tensorflow:Running against TensorRT version 0.0.0


In [0]:
graph_io.write_graph(trt_graph, root_path,
                     'trt_graph.pb', as_text=False)

'/content/gdrive/My Drive/USDA/Nano_Tensorflow/trt_graph.pb'